## Imports

In [1]:
import requests
from bs4 import BeautifulSoup 
import re
import pandas as pd 
import os 
import time
import datetime
import csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pandas.io.json import json_normalize
import csv
from pyquery import PyQuery as pq
from lxml import etree
import requests

## Web Scraping MetroLyrics

In [ ]:
sesamesongs = [] 

for x in range(1,11):
    newsong = {'Sesame Street': 'http://www.metrolyrics.com/sesame-street-alpage-' + str(x) + '.html'}
    sesamesongs.append(newsong)

In [ ]:
Allsongs = []

for l in sesamesongs:
    for key, value in l.items():
        
        print(value) ## To keep track of progress
        
        response = requests.get(value)
        doc = pq(response.content)       
        titles = doc('.title')

        
        for title in titles:
            if 'Sesame' in (title.attrib['title']):
                header = (title.attrib['title'])
                response_title = requests.get(title.attrib['href'])
                doc2 = pq(response_title.content)     
                verse = doc2('.verse')
            
                lyrics =  (verse.text())
            
                print(header) ## To keep track of progress
            
                newsong = {'Show' : key, 'Lyrics': lyrics, 'Song Name' : header } #### FIGURE THIS OUT!... Song Name' : header 
                Allsongs.append(newsong)
            
        
                time.sleep(4.5)
        

In [ ]:
Allsongs.to_csv('SesameSongs.csv')

## NLP - Analysis

### Initial Cleaning

In [2]:
SesameSongs = pd.read_csv('SesameSongs.csv').drop('Unnamed: 0',1)

In [3]:
SesameSongs

,Lyrics,Show,Song Name
0,"Polly Darton by a country fence, looking over ...",Sesame Street,Sesame Street 14 Carrot Love lyrics
1,Voice #1: I heard a bark. Go out and see what ...,Sesame Street,Sesame Street A Bark In The Dark lyrics
2,Did you ever wanna see what the birdies see?\n...,Sesame Street,Sesame Street A Birdie's Eye lyrics
3,There's a brand new baby at Almariz's house\nT...,Sesame Street,Sesame Street A Brand New Baby lyrics
4,"Okay everybody, I'm gonna tell you about some ...",Sesame Street,Sesame Street A Cat Had A Birthday lyrics
5,A face can be up\nA face can be down\nA face c...,Sesame Street,Sesame Street A Face lyrics
6,"Olivia: In the winter, springtime, and fall .....",Sesame Street,Sesame Street A Friend For All Seasons lyrics
7,Be kind to the river\nBe good to the trees\nSm...,Sesame Street,Sesame Street A Gift lyrics
8,"Henry: There's a hole in the bucket, dear Liza...",Sesame Street,Sesame Street A Hole In The Bucket lyrics
9,A girl named Dinah lived in South Carolina\nSh...,Sesame Street,Sesame Street A Letter To Ling Lu lyrics


In [4]:
SesameSongs_clean = SesameSongs.apply(lambda x: x.str.strip()).replace('', np.nan)

In [5]:
SesameSongs_clean = SesameSongs_clean.dropna().replace(r'\([^)]*\)','', regex=True).replace(r'\#\w*:','', regex=True).replace(r'\w*:','', regex=True)

In [503]:
SesameSongs_clean['Lyrics'][1]

"Voice  I heard a bark. Go out and see what it is.\nVoice  A bark?\nVoice  Go on and see.\nVoice  Okay, but it's awfully dark out here.\n\nVoice  What happened? I heard a bump.\nVoice  Oh, it's just me. I tripped over that broken chair.\nVoice  An old broken chair? Can't you take that to the dump?\nVoice  Not in the dark, I can't. Now leave me alone!  Sure is dark out here ...  Wait a minute? Did you hear a little, tiny bark?\nVoice  No, I didn't hear a bark.\nVoice  Well, I was sure I heard a ...\nVoice  Bark!\nVoices #1 and  YEEEOOOWWW!\n\n Can you give me directions to the park? Rrruff!"

### New Column: Word count

In [6]:
SesameSongs_clean['Word Count'] = SesameSongs_clean['Lyrics'].str.split().str.len()

### New Column: Non-English words removed

In [37]:
import nltk
words = set(nltk.corpus.words.words())

OnlyEng = []

for song in SesameSongs_clean['Lyrics']:
    sent = " ".join(w for w in nltk.wordpunct_tokenize(song) \
                    if w.lower() in words or not w.isalpha())
    OnlyEng.append(sent)
        

In [38]:
SesameSongs_clean['Only English Words'] = OnlyEng

### New Column: Number of unique English words used

In [11]:
Uniquecount = []

for song in SesameSongs_clean['Only English Words']:
    uniqueWords = len(set(nltk.wordpunct_tokenize(song)))
    
    Uniquecount.append(uniqueWords)

# uniqueWords = list(set(" ".join(SesameSongs_clean['Only English Words']).lower().split(" ")))

SesameSongs_clean['Unique Eng Words'] = Uniquecount

### New Column: Proportion of total words that are unique.

In [12]:
Uniqueprop = []

for song in SesameSongs_clean['Only English Words']:
    unique = len(set(nltk.wordpunct_tokenize(song)))
    prop = round(unique / len(nltk.wordpunct_tokenize(song)),3)
    
    Uniqueprop.append(prop)

In [13]:
SesameSongs_clean['Prop of unique words'] = Uniqueprop

### New Column: Count of how many numbers are mentioned

In [14]:
numlist = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','one','two','three','four','five','six','seven','eight','nine','ten']

Numcount = []
counter = 0

for song in SesameSongs_clean['Lyrics']:
    counter = 0
    for w in nltk.wordpunct_tokenize(song):
        if w in numlist:
            counter += 1
    
    Numcount.append(counter)
    

SesameSongs_clean['How many numbers?'] = Numcount

### New Column: Proportion of song that starts with the letter that appears most often

In [15]:
from string import ascii_lowercase

nonnumeric = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','.',',','!','?','...','#','"',"'",')','-']

listodic = []
letterprop = []

d = {}

for song in SesameSongs_clean['Lyrics']:
    d = {}
    x = nltk.wordpunct_tokenize(song)
    for word in x:
        word = word.lower()
        if word not in nonnumeric:
            if word[0] not in d:
                d[word[0]] = 1
            
            else:
                d[word[0]] += 1
                  
    v=list(d.values())
    most = (max(v))
    
    letterprop.append(round(most/len(nltk.wordpunct_tokenize(song)),3))


In [16]:
SesameSongs_clean['Letter Proportion'] = letterprop

### New Column: Number of times "you" "yours "your" is used.

In [ ]:
interactive = ['you','your',"y'all",'you all','our','we']
rest = ['I','he','she','their','they','them','hers','his','theirs']

subjects = []


youcounter = 0
restcounter = 0


for song in SesameSongs_clean['Only English Words']:
    youcounter = 0
    restcounter = 0
    for w in nltk.wordpunct_tokenize(song):
        if w in interactive:
            youcounter += 1
        if w in rest:
            restcounter +=1
    
    if (restcounter+youcounter) != 0:
        youprop = (youcounter/(youcounter+restcounter))
        
    else:
        youprop = 0
    
    subjects.append(youprop)

In [ ]:
SesameSongs_clean['Interactive language prop'] = subjects

### New Columns: Polarity and Subjectivity

In [19]:
from textblob import TextBlob

In [20]:
SesameSongs_clean = SesameSongs_clean.reset_index(drop = True)

In [21]:
subjectivity = []
polarity = []

for word in range(len(SesameSongs_clean['Lyrics'])):
    w = str(SesameSongs_clean["Lyrics"][word])
    d = list(TextBlob(w).sentiment)
    polarity.append(d[0])
    subjectivity.append(d[1])

In [22]:
SesameSongs_clean['Polarity'] = polarity

In [23]:
SesameSongs_clean['Subjectivity'] = subjectivity

### New Column: Number of words outside Baladan word list

**Creating columns to identify how many "fringe" words are used (i.e. words that are less common in adult speech).** <br>
Baladan 347 words account for 75% of adult speech. 

##### **Creating Baladan List**

In [24]:
baladinpd = pd.read_csv('Baladin_WordList.csv')

In [25]:
baladin = []

import csv
with open('Baladin_WordList.csv', 'r') as f:
    reader = csv.reader(f)
    baladinlist = list(reader)

for listy in baladinlist:
    for word in listy:
        if word != '':
            baladin.append(word)

In [26]:
len(baladin)

347

##### **Counting number of words outside of the list**

In [27]:
fringewords = []

fringecounts = []
fringeprop = []
    

for song in SesameSongs_clean['Only English Words']:
    fringewords = []
    for w in nltk.wordpunct_tokenize(song):
        w = w.lower()
        if w not in baladin:
            if w not in nonnumeric:
                fringewords.append(w)
    
    
    fringey = (len(set(fringewords)))
    uniques = len(set(nltk.wordpunct_tokenize(song)))
    
    fringecounts.append(fringey)   
    fringeprop.append(round((fringey/uniques), 3)) 

In [28]:
SesameSongs_clean['Adult fringe count'] = fringecounts

In [29]:
SesameSongs_clean['Adult fringe prop'] = fringeprop

### New Column: Number of words outside Marvin word list

**Creating columns to identify how many "fringe" words are used (i.e. words that are less in toddler speech).** <br>
Baladan 333 words account for majority of pre-schooler core speech.

##### **Creating Marvin list**

In [30]:
marvinpd = pd.read_csv('Marvin_WordList.csv')

In [31]:
marvin = []

import csv
with open('Marvin_WordList.csv', 'r') as f:
    reader = csv.reader(f)
    marvinlist = list(reader)

for listy in marvinlist:
    for word in listy:
        if len(word)>0:
            if word[0] not in numlist:
                marvin.append(word)

In [32]:
marvin = (set(marvin))

##### **Counting number of words outside of the list**

In [33]:
mfringewords = []

mfringecounts = []
mfringeprop = []
    

for song in SesameSongs_clean['Only English Words']:
    mfringewords = []
    for w in nltk.wordpunct_tokenize(song):
        w = w.lower()
        if w not in marvin:
            if w not in nonnumeric:
                mfringewords.append(w)
    
    
    fringey = (len(set(mfringewords)))
    uniques = len(set(nltk.wordpunct_tokenize(song)))
    
    mfringecounts.append(fringey)   
    mfringeprop.append(round((fringey/uniques), 3)) 

In [34]:
SesameSongs_clean['PreK fringe count'] = mfringecounts

In [35]:
SesameSongs_clean['PreK fringe prop'] = mfringeprop

## Cluster Modeling

#### Preprocessing

In [36]:
SesameSongs_clean

,Lyrics,Show,Song Name,Word Count,Only English Words,Unique Eng Words,Prop of unique words,How many numbers?,Letter Proportion,Interactive language prop,Polarity,Subjectivity,Adult fringe count,Adult fringe prop,PreK fringe count,PreK fringe prop
0,"Polly Darton by a country fence, looking over ...",Sesame Street,Sesame Street 14 Carrot Love lyrics,477,"by a country fence , looking over 13 , laid ou...",174,0.285,47,0.073,0.452381,0.310672,0.602455,41,0.236,44,0.253
1,Voice I heard a bark. Go out and see what it ...,Sesame Street,Sesame Street A Bark In The Dark lyrics,116,Voice I a bark . Go out and see what it is . V...,64,0.451,1,0.085,0.300000,0.054687,0.497980,17,0.266,19,0.297
2,Did you ever wanna see what the birdies see?\n...,Sesame Street,Sesame Street A Birdie's Eye lyrics,61,Did you ever see what the see ? Well I do ! Di...,34,0.479,0,0.152,0.571429,0.232000,0.703556,5,0.147,8,0.235
3,There's a brand new baby at Almariz's house\nT...,Sesame Street,Sesame Street A Brand New Baby lyrics,61,There ' s a brand new baby at ' s house There ...,38,0.528,0,0.234,0.000000,0.189719,0.431126,11,0.289,14,0.368
4,"Okay everybody, I'm gonna tell you about some ...",Sesame Street,Sesame Street A Cat Had A Birthday lyrics,310,"everybody , I ' m tell you about some birthday...",105,0.266,0,0.108,0.428571,0.348750,0.578810,31,0.295,32,0.305
5,A face can be up\nA face can be down\nA face c...,Sesame Street,Sesame Street A Face lyrics,148,A face can be up A face can be down A face can...,54,0.342,0,0.108,1.000000,-0.027778,0.381944,21,0.389,19,0.352
6,"In the winter, springtime, and fall ...\nIn t...",Sesame Street,Sesame Street A Friend For All Seasons lyrics,125,"In the winter , springtime , and fall ... In t...",57,0.373,0,0.120,0.600000,0.306641,0.737500,23,0.404,21,0.368
7,Be kind to the river\nBe good to the trees\nSm...,Sesame Street,Sesame Street A Gift lyrics,111,Be kind to the river Be good to the Smile at t...,58,0.460,0,0.260,0.800000,0.457143,0.542857,26,0.448,30,0.517
8,"There's a hole in the bucket, dear Liza, dear...",Sesame Street,Sesame Street A Hole In The Bucket lyrics,450,"There ' s a hole in the bucket , dear , dear ,...",82,0.149,0,0.142,0.000000,-0.111133,0.407828,31,0.378,35,0.427
9,A girl named Dinah lived in South Carolina\nSh...,Sesame Street,Sesame Street A Letter To Ling Lu lyrics,134,A girl lived in South She wrote a letter to a ...,68,0.511,0,0.182,0.400000,0.400000,0.480000,27,0.397,32,0.471


** Selecting inputs for the model **

In [321]:
S_inputs = SesameSongs_clean.filter(items = [
#        'Word Count', 
#        'Unique Eng Words',
       'Prop of unique words',  
#     'How many numbers?',
#     'Letter Proportion',
    'Interactive language prop',
    'Adult fringe prop',
    'PreK fringe prop',
#     'PreK fringe count',
#     'Adult fringe count'
])

In [322]:
S_inputs

,Prop of unique words,Interactive language prop,Adult fringe prop,PreK fringe prop
0,0.285,0.452381,0.236,0.253
1,0.451,0.300000,0.266,0.297
2,0.479,0.571429,0.147,0.235
3,0.528,0.000000,0.289,0.368
4,0.266,0.428571,0.295,0.305
5,0.342,1.000000,0.389,0.352
6,0.373,0.600000,0.404,0.368
7,0.460,0.800000,0.448,0.517
8,0.149,0.000000,0.378,0.427
9,0.511,0.400000,0.397,0.471


** Scaling **

In [270]:
from sklearn.preprocessing import scale

In [178]:
S_inputs['Word Count'] = scale(S_inputs['Word Count'])

/Users/vicky/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [179]:
S_inputs['How many numbers?'] = scale(S_inputs['How many numbers?'])

/Users/vicky/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [181]:
S_inputs['PreK fringe count'] = scale(S_inputs['PreK fringe count'])

/Users/vicky/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [182]:
S_inputs['Adult fringe count'] = scale(S_inputs['Adult fringe count'])

/Users/vicky/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [323]:
S_inputs

,Prop of unique words,Interactive language prop,Adult fringe prop,PreK fringe prop
0,0.285,0.452381,0.236,0.253
1,0.451,0.300000,0.266,0.297
2,0.479,0.571429,0.147,0.235
3,0.528,0.000000,0.289,0.368
4,0.266,0.428571,0.295,0.305
5,0.342,1.000000,0.389,0.352
6,0.373,0.600000,0.404,0.368
7,0.460,0.800000,0.448,0.517
8,0.149,0.000000,0.378,0.427
9,0.511,0.400000,0.397,0.471


#### Running Hierarchical Aggllomerative Clustering model

In [324]:
from sklearn.cluster import AgglomerativeClustering

In [325]:
Aclus = AgglomerativeClustering(n_clusters = 3,linkage = 'ward')
Aclus.fit(S_inputs)

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=None, linkage='ward', memory=None, n_clusters=3,
            pooling_func=<function mean at 0x114ba6840>)

In [479]:
S_inputs

,Prop of unique words,Interactive language prop,Adult fringe prop,PreK fringe prop
0,0.285,0.452381,0.236,0.253
1,0.451,0.300000,0.266,0.297
2,0.479,0.571429,0.147,0.235
3,0.528,0.000000,0.289,0.368
4,0.266,0.428571,0.295,0.305
5,0.342,1.000000,0.389,0.352
6,0.373,0.600000,0.404,0.368
7,0.460,0.800000,0.448,0.517
8,0.149,0.000000,0.378,0.427
9,0.511,0.400000,0.397,0.471


In [326]:
p = Aclus.fit_predict(S_inputs)

In [502]:
# newsong = np.array([0.28,0.8,0.34,0.5])
# newsong2 = np.reshape(newsong,(1,-1))

# Aclus.predict(S_inputs.iloc[0:2])

In [327]:
Aclus.get_params()

{'affinity': 'euclidean',
 'compute_full_tree': 'auto',
 'connectivity': None,
 'linkage': 'ward',
 'memory': None,
 'n_clusters': 3,
 'pooling_func': <function numpy.core.fromnumeric.mean>}

In [328]:
SesameSongs_clustered = SesameSongs_clean

In [329]:
SesameSongs_clustered['Cluster'] = p

In [330]:
SesameSongs_clustered

,Lyrics,Show,Song Name,Word Count,Only English Words,Unique Eng Words,Prop of unique words,How many numbers?,Letter Proportion,Interactive language prop,Polarity,Subjectivity,Adult fringe count,Adult fringe prop,PreK fringe count,PreK fringe prop,Cluster
0,"Polly Darton by a country fence, looking over ...",Sesame Street,Sesame Street 14 Carrot Love lyrics,477.0,"by a country fence , looking over 13 , laid ou...",174.0,0.285,47.0,0.073,0.452381,0.310672,0.602455,41.0,0.236,44.0,0.253,1
1,Voice I heard a bark. Go out and see what it ...,Sesame Street,Sesame Street A Bark In The Dark lyrics,116.0,Voice I a bark . Go out and see what it is . V...,64.0,0.451,1.0,0.085,0.300000,0.054687,0.497980,17.0,0.266,19.0,0.297,0
2,Did you ever wanna see what the birdies see?\n...,Sesame Street,Sesame Street A Birdie's Eye lyrics,61.0,Did you ever see what the see ? Well I do ! Di...,34.0,0.479,0.0,0.152,0.571429,0.232000,0.703556,5.0,0.147,8.0,0.235,1
3,There's a brand new baby at Almariz's house\nT...,Sesame Street,Sesame Street A Brand New Baby lyrics,61.0,There ' s a brand new baby at ' s house There ...,38.0,0.528,0.0,0.234,0.000000,0.189719,0.431126,11.0,0.289,14.0,0.368,0
4,"Okay everybody, I'm gonna tell you about some ...",Sesame Street,Sesame Street A Cat Had A Birthday lyrics,310.0,"everybody , I ' m tell you about some birthday...",105.0,0.266,0.0,0.108,0.428571,0.348750,0.578810,31.0,0.295,32.0,0.305,1
5,A face can be up\nA face can be down\nA face c...,Sesame Street,Sesame Street A Face lyrics,148.0,A face can be up A face can be down A face can...,54.0,0.342,0.0,0.108,1.000000,-0.027778,0.381944,21.0,0.389,19.0,0.352,2
6,"In the winter, springtime, and fall ...\nIn t...",Sesame Street,Sesame Street A Friend For All Seasons lyrics,125.0,"In the winter , springtime , and fall ... In t...",57.0,0.373,0.0,0.120,0.600000,0.306641,0.737500,23.0,0.404,21.0,0.368,1
7,Be kind to the river\nBe good to the trees\nSm...,Sesame Street,Sesame Street A Gift lyrics,111.0,Be kind to the river Be good to the Smile at t...,58.0,0.460,0.0,0.260,0.800000,0.457143,0.542857,26.0,0.448,30.0,0.517,1
8,"There's a hole in the bucket, dear Liza, dear...",Sesame Street,Sesame Street A Hole In The Bucket lyrics,450.0,"There ' s a hole in the bucket , dear , dear ,...",82.0,0.149,0.0,0.142,0.000000,-0.111133,0.407828,31.0,0.378,35.0,0.427,0
9,A girl named Dinah lived in South Carolina\nSh...,Sesame Street,Sesame Street A Letter To Ling Lu lyrics,134.0,A girl lived in South She wrote a letter to a ...,68.0,0.511,0.0,0.182,0.400000,0.400000,0.480000,27.0,0.397,32.0,0.471,0


#### EDA on each cluster

In [331]:
SesameSongs_clustered['Cluster'].value_counts()

0    289
1    188
2    147
Name: Cluster, dtype: int64

In [559]:
cluster_analysis = SesameSongs_clustered.groupby(['Cluster'])[
    'Word Count', 
    'Unique Eng Words',
    'Prop of unique words',  
    'How many numbers?',
    'Letter Proportion',
    'Interactive language prop',
    'Adult fringe prop',
    'PreK fringe prop',
    'PreK fringe count',
    'Adult fringe count'
].mean()

In [560]:
cluster_analysis

,Word Count,Unique Eng Words,Prop of unique words,How many numbers?,Letter Proportion,Interactive language prop,Adult fringe prop,PreK fringe prop,PreK fringe count,Adult fringe count
Cluster,,,,,,,,,,
0,204.013841,88.771626,0.408170,3.982699,0.160308,0.171277,0.388204,0.413713,36.816609,34.972318
1,202.053191,92.930851,0.424356,3.308511,0.142436,0.600009,0.366947,0.393096,36.393617,34.244681
2,177.578231,70.761905,0.380218,2.319728,0.174456,0.922756,0.349905,0.376782,26.782313,24.986395


**It seems like generally... ** <br>
0. Has the highest proportion of new words.
1. Have the most words. 
2. Has the highest proportion of words that break the fourth wall



## LDA on each cluster

** Seperating each cluster into own dataframe **

In [340]:
cluster0 = SesameSongs_clustered.loc[SesameSongs_clustered['Cluster'] == 0]

In [341]:
cluster1 = SesameSongs_clustered.loc[SesameSongs_clustered['Cluster'] == 1]

In [342]:
cluster2 = SesameSongs_clustered.loc[SesameSongs_clustered['Cluster'] == 2]

** LDA on each cluster **

#### Cluster 0

In [344]:
from __future__ import print_function
# gensim
from gensim import corpora, models, similarities, matutils
# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
# logging for gensim (set to INFO)
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer 


In [432]:
Songs0 = cluster0['Only English Words'].tolist()

In [433]:


vectorizer = CountVectorizer(ngram_range=(1,1), stop_words="english")

# call `fit` to build the vocabulary
vectorizer.fit(Songs0)

x = vectorizer.fit_transform(Songs0)
# # then, use `get_feature_names` to return the tokens
# print(vectorizer.get_feature_names())

counts = vectorizer.transform(Songs0).transpose()


In [397]:
counts.shape

(2948, 289)

In [423]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components= 3, max_iter = 15, random_state=0, 
                                n_jobs=-1, learning_method='batch')

In [424]:
topics = lda.fit_transform(x)

In [425]:
lda.components_.shape

(3, 2948)

In [426]:
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [427]:
pyLDAvis.sklearn.prepare(lda, x, vectorizer)

/Users/vicky/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      37.803077        1       1 -0.067001  0.096410
0      33.313057        1       2 -0.055312 -0.102755
1      28.883866        1       3  0.122312  0.006345, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
1363  Default  170.000000          la  170.000000  30.0000  30.0000
1218  Default   97.000000        honk   97.000000  29.0000  29.0000
2912  Default   90.000000       write   90.000000  28.0000  28.0000
653   Default   80.000000        dear   80.000000  27.0000  27.0000
660   Default   56.000000         dee   56.000000  26.0000  26.0000
716   Default   52.000000         dog   52.000000  25.0000  25.0000
1150  Default   97.000000       happy   97.000000  24.0000  24.0000
144   Default   81.000000        baby   81.000000  23.0000  23.0000
1488  Default   54.000000         mad   54.000000  22.0000  22.0000
236   Default   69.000000        bird   69.000000  21.0000  21.0000
2703  Default   36.000000  typewriter   36.000000  20.0000  20.0000
1187  Default   41.000000       henry   41.000000  19.0000  19.0000
1819  Default   41.000000       piper   41.000000  18.0000  18.0000
2216  Default  109.000000        sing  109.000000  17.0000  17.0000
2607  Default   46.000000       times   46.000000  16.0000  16.0000
1441  Default  192.000000      little  192.000000  15.0000  15.0000
1586  Default   62.000000     monster   62.000000  14.0000  14.0000
1639  Default   28.000000      nearly   28.000000  13.0000  13.0000
2063  Default   32.000000         sad   32.000000  12.0000  12.0000
2326  Default   35.000000       sound   35.000000  11.0000  11.0000
528   Default   51.000000       comes   51.000000  10.0000  10.0000
1127  Default   32.000000        hair   32.000000   9.0000   9.0000
1240  Default   45.000000         huh   45.000000   8.0000   8.0000
1215  Default   27.000000       homer   27.000000   7.0000   7.0000
967   Default   27.000000      follow   27.000000   6.0000   6.0000
562   Default   65.000000       count   65.000000   5.0000   5.0000
2895  Default   42.000000        word   42.000000   4.0000   4.0000
286   Default   21.000000         bop   21.000000   3.0000   3.0000
1010  Default   24.000000         fur   24.000000   2.0000   2.0000
1964  Default   36.000000        read   36.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
1678   Topic3   18.112020        nose   23.374863   0.9868  -5.8131
1002   Topic3   21.605854        frog   29.461245   0.9318  -5.6367
1200   Topic3   11.461371         hip   13.798925   1.0563  -6.2706
562    Topic3   39.009966       count   65.034803   0.7308  -5.0458
2791   Topic3   11.310025        wall   13.835633   1.0403  -6.2839
262    Topic3   22.850413        blue   37.365244   0.7501  -5.5807
2578   Topic3   38.593770       think   79.282161   0.5220  -5.0565
2925   Topic3   49.952424        yeah  118.943074   0.3743  -4.7986
93     Topic3   16.317638     anybody   23.424755   0.8803  -5.9174
1684   Topic3   32.323599      number   66.250687   0.5242  -5.2338
1502   Topic3   29.352784         man   58.429092   0.5535  -5.3302
1431   Topic3   71.103009        like  246.834504  -0.0027  -4.4455
1360   Topic3   51.015435        know  157.466871   0.1148  -4.7775
1077   Topic3   48.424969         got  158.283716   0.0575  -4.8296
723    Topic3   45.382112         don  150.639060   0.0421  -4.8945
1664   Topic3   25.426542       night   56.777934   0.4385  -5.4738
1696   Topic3   57.548349          oh  297.268326  -0.4001  -4.6570
1697   Topic3   24.983863         old   57.494592   0.4084  -5.4914
1469   Topic3   40.992811        love  182.557417  -0.2518  -4.9962
1336   Topic3   41.416837        just  193.370441  -0.2990  -4.9859
1416   Topic3   30.838728      letter  100.342220   0.0621  -5.2809
1190   Topic3  

In [415]:
from nltk import FreqDist

Songs0all = []

for song in Songs0:
    for word in song:
        Songs0all.append(word)
    
    
pop0words = nltk.probability.FreqDist(Songs0)

print(pop0words)

v=list(pop0words.values())
sort = (sorted(v))

from operator import itemgetter
# listy = sorted(pop0words.items(), key=itemgetter(reverse=True))

# listy

<FreqDist with 286 samples and 289 outcomes>


#### Cluster 1

In [ ]:
Cluster1 = Cluster1.replace('.', '')

In [457]:
Songs1 = cluster1['Only English Words'].tolist()

In [458]:
Songs1 = filter(lambda i:not(type(i) is str), Songs1)

In [461]:
vectorizer = CountVectorizer(ngram_range=(1,1), stop_words="english")

# call `fit` to build the vocabulary
vectorizer.fit(Songs1)

x = vectorizer.fit_transform(Songs1)
# # then, use `get_feature_names` to return the tokens
# print(vectorizer.get_feature_names())

counts = vectorizer.transform(Songs1).transpose()

#### Cluster 2

In [439]:
Songs2 = cluster2['Only English Words'].tolist()

In [440]:
vectorizer = CountVectorizer(ngram_range=(1,1), stop_words="english")

# call `fit` to build the vocabulary
vectorizer.fit(Songs2)

x = vectorizer.fit_transform(Songs2)
# # then, use `get_feature_names` to return the tokens
# print(vectorizer.get_feature_names())

counts = vectorizer.transform(Songs2).transpose()

In [441]:
counts.shape

(1577, 147)

In [442]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components= 3, max_iter = 15, random_state=0, 
                                n_jobs=-1, learning_method='batch')

In [443]:
topics = lda.fit_transform(x)

In [445]:
pyLDAvis.sklearn.prepare(lda, x, vectorizer)

/Users/vicky/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      38.337086        1       1 -0.062364  0.124407
0      35.786610        1       2 -0.084850 -0.112352
2      25.876304        1       3  0.147214 -0.012054, topic_info=     Category       Freq        Term       Total  loglift  logprob
term                                                              
722   Default  94.000000          la   94.000000  30.0000  30.0000
1510  Default  45.000000          wi   45.000000  29.0000  29.0000
940   Default  51.000000         pat   51.000000  28.0000  28.0000
1081  Default  38.000000         run   38.000000  27.0000  27.0000
1126  Default  45.000000       shake   45.000000  26.0000  26.0000
60    Default  70.000000        baby   70.000000  25.0000  25.0000
1512  Default  40.000000      wiggle   40.000000  24.0000  24.0000
1050  Default  30.000000       rhyme   30.000000  23.0000  23.0000
1313  Default  26.000000       swish   26.000000  22.0000  22.0000
853   Default  47.000000        moon   47.000000  21.0000  21.0000
1560  Default  32.000000         yip   32.000000  20.0000  20.0000
634   Default  23.000000        hole   23.000000  19.0000  19.0000
130   Default  24.000000    boarding   24.000000  18.0000  18.0000
1033  Default  34.000000       reach   34.000000  17.0000  17.0000
652   Default  29.000000       house   29.000000  16.0000  16.0000
778   Default  19.000000         log   19.000000  15.0000  15.0000
26    Default  19.000000       alive   19.000000  14.0000  14.0000
94    Default  27.000000        beet   27.000000  13.0000  13.0000
1214  Default  27.000000    somebody   27.000000  12.0000  12.0000
174   Default  17.000000        bump   17.000000  11.0000  11.0000
1317  Default  25.000000       takin   25.000000  10.0000  10.0000
1108  Default  28.000000         sea   28.000000   9.0000   9.0000
980   Default  56.000000        play   56.000000   8.0000   8.0000
1423  Default  30.000000       turns   30.000000   7.0000   7.0000
296   Default  21.000000    counting   21.000000   6.0000   6.0000
1215  Default  17.000000     someday   17.000000   5.0000   5.0000
505   Default  30.000000        frog   30.000000   4.0000   4.0000
295   Default  46.000000       count   46.000000   3.0000   3.0000
1539  Default  19.000000        worm   19.000000   2.0000   2.0000
867   Default  33.000000       music   33.000000   1.0000   1.0000
...       ...        ...         ...         ...      ...      ...
1495   Topic3   5.256333       weigh    5.836401   1.2472  -6.1616
392    Topic3   5.252118     driving    5.836616   1.2463  -6.1624
1330   Topic3   4.426439       teeny    5.006438   1.2287  -6.3334
1494   Topic3   4.426439       weeny    5.006438   1.2287  -6.3334
625    Topic3   4.426436        hill    5.006438   1.2287  -6.3334
53     Topic3   4.426432  automobile    5.006438   1.2287  -6.3334
1395   Topic3   4.426432      travel    5.006438   1.2287  -6.3334
652    Topic3  25.197638       house   29.224937   1.2036  -4.5943
296    Topic3  17.921505    counting   21.764170   1.1576  -4.9350
1517   Topic3   7.767268        wing    9.197553   1.1828  -5.7711
1108   Topic3  21.970771         sea   28.512686   1.0912  -4.7313
560    Topic3   7.750481        grew    9.198410   1.1806  -5.7732
155    Topic3  16.060716     breathe   21.830826   1.0449  -5.0446
295    Topic3  28.177059       count   46.486465   0.8512  -4.4825
505    Topic3  19.374169        frog   30.356574   0.9028  -4.8571
440    Topic3  10.762686      family   14.269604   1.0698  -5.4449
720    Topic3  37.084277        know   85.215471   0.5199  -4.2078
908    Topic3  50.121645          oh  135.015345   0.3609  -3.9066
549    Topic3  34.845660         got   80.095808   0.5195  -4.2701
214    Topic3  10.243136       chair   13.418652   1.0818  -5.4944
110    Topic3  26.246440         big   56.972823   0.5768  -4.5535
791    Topic3  25.767853        love   70.095041   0.3511  -4.5719
62

## Song writing

In [615]:
mysong = 'When Im feeling mad or sad I Follow my imagination till things arent so bad Try these things I like to do, And before you know it youll be joyful too. I Listen to the sounds of a baby bird Or Grab a book and read every word I Bop my head to a song’s little rhymes And sing along to pass the time I think of a place Id like to visit, And in my mind, its quite exquisite! I Honk the horn of an blue car I’m nearly there, I’m not so far! I Follow my dog to his favorite park, And befriend a furry monster when it gets dark, I Count the Hairs on the head of a dear pied piper Or write happy words on a typewriter! These are the things I like to do, I think youll love to do them too!'

** Baladin score **

In [622]:
fringewords = []

fringecounts = []
fringeprop = []
    
for w in nltk.wordpunct_tokenize(mysong):
    w = w.lower()
    if w not in baladin:
        if w not in nonnumeric:
            fringewords.append(w)
    

In [623]:

wordlist = fringewords.remove('youll')
wordlist = fringewords.remove('arent')

fringey = (len(set(fringewords)))
uniques = len(set(nltk.wordpunct_tokenize(mysong)))
    
fringecounts.append(fringey)   
fringeprop.append(round((fringey/uniques), 3)) 

In [624]:
fringeprop

[0.5]

In [625]:
fringecounts

[50]

In [626]:
balwordlist = set(fringewords)

In [627]:
balwordlist

{'along',
 'baby',
 'befriend',
 'bird',
 'blue',
 'book',
 'bop',
 'count',
 'dark',
 'dog',
 'exquisite',
 'far',
 'favorite',
 'feeling',
 'follow',
 'furry',
 'grab',
 'hairs',
 'happy',
 'head',
 'honk',
 'horn',
 'id',
 'im',
 'imagination',
 'joyful',
 'listen',
 'm',
 'mad',
 'monster',
 'nearly',
 'park',
 'pass',
 'pied',
 'piper',
 'read',
 'rhymes',
 's',
 'sad',
 'sing',
 'song',
 'sounds',
 'till',
 'typewriter',
 'visit',
 'word',
 'words',
 'write',
 'youll',
 '’'}

** Marvin score **

In [639]:
mfringewords = []

mfringecounts = []
mfringeprop = []
    


for w in nltk.wordpunct_tokenize(mysong):
    w = w.lower()
    if w not in marvin:
        if w not in nonnumeric:
            mfringewords.append(w)
    
    

In [640]:

fringey = (len(set(mfringewords)))
uniques = len(set(nltk.wordpunct_tokenize(mysong)))
    
mfringecounts.append(fringey)   
mfringeprop.append(round((fringey/uniques), 3)) 

In [641]:
set(mfringewords)

{'a',
 'along',
 'arent',
 'befriend',
 'book',
 'bop',
 'count',
 'dark',
 'dear',
 'every',
 'exquisite',
 'far',
 'favorite',
 'feeling',
 'follow',
 'furry',
 'grab',
 'hairs',
 'happy',
 'honk',
 'horn',
 'i',
 'id',
 'im',
 'imagination',
 'its',
 'joyful',
 'listen',
 'love',
 'm',
 'mad',
 'mind',
 'monster',
 'nearly',
 'park',
 'pass',
 'pied',
 'piper',
 'place',
 'quite',
 'read',
 'rhymes',
 's',
 'sad',
 'sing',
 'song',
 'sounds',
 'till',
 'typewriter',
 'visit',
 'word',
 'words',
 'write',
 'youll',
 '’'}

In [642]:
mfringecounts

[55]

In [643]:
mfringeprop

[0.55]